# Using Uniform Manifold Approximation and Projection (UMAP) to Differentiate Subset Populations
#### In this Jupyter Notebook, we will be following the standard setup for a UMAP environment as well as perform a UMAP analysis on an example data set.
***

#### Import Tools for UMAP: 

In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

The main tools for UMAP are those that we have already seen in class - numpy, seaborn, and pandas. It's cool to see the content we're taught in class used for these sorts of high-level analyses! Without all of these packages installed, the UMAP package won't be able to run! 

The sklearn packages are also required to run UMAP, they are used primarily in the preprocessing of the data.

We also need to make sure that the conda environment has the UMAP package installed, to do this - go to your terminal and do:
```
source activate <conda environment/kernel>
conda install -c conda-forge umap-learn
```